# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pacific grove,36.6177,-121.9166,24.14,84,39,2.24,US,1723075290
1,1,codrington,-38.2667,141.9667,11.95,65,100,9.83,AU,1723075292
2,2,boyalik,41.2634,28.6314,21.51,76,0,2.95,TR,1723075293
3,3,ushuaia,-54.8000,-68.3000,0.81,75,0,1.54,AR,1723075294
4,4,uturoa,-16.7333,-151.4333,26.14,80,16,4.88,PF,1723075295


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,  
    scale = 1,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_weather_df = city_data_df[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27) & (city_data_df["Humidity"]<90) & (city_data_df["Humidity"]>40)]
# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

# Display sample data
filtered_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pacific grove,36.6177,-121.9166,24.14,84,39,2.24,US,1723075290
2,2,boyalik,41.2634,28.6314,21.51,76,0,2.95,TR,1723075293
4,4,uturoa,-16.7333,-151.4333,26.14,80,16,4.88,PF,1723075295
5,5,port mathurin,-19.6833,63.4167,22.23,75,8,5.17,MU,1723075296
8,8,coruripe,-10.1256,-36.1756,22.14,84,6,3.12,BR,1723075300
...,...,...,...,...,...,...,...,...,...,...
571,571,seaside,36.6111,-121.8516,24.10,83,0,4.63,US,1723076016
572,572,trincomalee,8.5711,81.2335,26.93,80,83,6.62,LK,1723076017
573,573,silver city,32.7701,-108.2803,24.24,57,88,0.89,US,1723076019
575,575,montrose,38.4166,-108.2673,25.76,71,88,2.58,US,1723076021


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\44rul\AppData\Local\Temp\ipykernel_30968\3494836535.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,pacific grove,US,36.6177,-121.9166,84,
2,boyalik,TR,41.2634,28.6314,76,
4,uturoa,PF,-16.7333,-151.4333,80,
5,port mathurin,MU,-19.6833,63.4167,75,
8,coruripe,BR,-10.1256,-36.1756,84,
...,...,...,...,...,...,...
571,seaside,US,36.6111,-121.8516,83,
572,trincomalee,LK,8.5711,81.2335,80,
573,silver city,US,32.7701,-108.2803,57,
575,montrose,US,38.4166,-108.2673,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "api_key": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pacific grove - nearest hotel: Pacific Grove Inn
boyalik - nearest hotel: Durusu Club Airport Hotel
uturoa - nearest hotel: Hawaiki Nui hotel
port mathurin - nearest hotel: Escale Vacances
coruripe - nearest hotel: No hotel found
bel ombre - nearest hotel: Villa La Cachette
west island - nearest hotel: Cocos Village Bungalows
san antonio de pale - nearest hotel: No hotel found
ponta delgada - nearest hotel: Hotel Talisman
avola - nearest hotel: L'Ancora
nova sintra - nearest hotel: Residência Ka Dencho
albany - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
avarua - nearest hotel: Paradise Inn
yellowknife - nearest hotel: No hotel found
san miguel - nearest hotel: Continental Plaza Hotel
ribeira grande - nearest hotel: Hospedaria JSF
porto novo - nearest hotel: Hôtel 6500 CFA
guarapari - nearest hotel: Osmar Hotel
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
moroni - nearest hotel: Ville de Moroni
horta - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
0,pacific grove,US,36.6177,-121.9166,84,Pacific Grove Inn
2,boyalik,TR,41.2634,28.6314,76,Durusu Club Airport Hotel
4,uturoa,PF,-16.7333,-151.4333,80,Hawaiki Nui hotel
5,port mathurin,MU,-19.6833,63.4167,75,Escale Vacances
8,coruripe,BR,-10.1256,-36.1756,84,No hotel found
...,...,...,...,...,...,...
571,seaside,US,36.6111,-121.8516,83,Embassy Suites
572,trincomalee,LK,8.5711,81.2335,80,Summer set inn
573,silver city,US,32.7701,-108.2803,57,Murray Hotel
575,montrose,US,38.4166,-108.2673,71,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols = ["Hotel Name", "Country"],
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,  
    scale = 1,
    color = "City"
 )

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)